# Нейронные сети

## Персептрон

Персептрон — одна из простейших архитектур ИНС, изобретенная в 1957 году Фрэнком Розенблаттом .Он основан на несколько ином искусственном нейроне (см. рис. 10-4 ), называемом пороговым логическим блоком (TLU), а иногда и линейным пороговым блоком (LTU).Входные и выходные данные представляют собой числа (вместо двоичных значений включения/выключения), и каждому входному соединению присвоен вес. TLU сначала вычисляет линейную функцию своих входов: z = w 1 x 1 + w 2 x 2 + ⋯ + w n x n + b = w ⊺ x + b . Затем к результату применяется ступенчатая функция : h w ( x ) = Step( z ). Таким образом, это почти похоже на логистическую регрессию, за исключением того, что вместо логистической функции используется ступенчатая функция (Глава 4 ). Как и в логистической регрессии, параметрами модели являются входные веса w и член смещения b .

Scikit-Learn предоставляет Perceptron класс, который можно использовать практически так, как и следовало ожидать, например, для набора данных ирисов:

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = (iris.target == 0)  

per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

X_new = [[2, 0.5], [3, 1]]
y_pred = per_clf.predict(X_new)  

алгоритм обучения перцептрона сильно напоминает стохастический градиентный спуск. Фактически, Perceptron класс Scikit-Learn эквивалентен использованию SGDClassifier со следующими гиперпараметрами: loss="perceptron", learning_rate="constant", eta0=1(скорость обучения) и penalty=None(без регуляризации ).

## Регрессионные MLP

Во-первых, MLP можно использовать для задач регрессии.Если вы хотите спрогнозировать одно значение (например, цену дома с учетом многих его характеристик), то вам нужен всего лишь один выходной нейрон: его выходом будет прогнозируемое значение. Для многомерной регрессии (т. е. для прогнозирования нескольких значений одновременно) вам нужен один выходной нейрон на каждое выходное измерение. Например, чтобы найти центр объекта на изображении, вам необходимо спрогнозировать двумерные координаты, поэтому вам нужны два выходных нейрона. Если вы также хотите разместить вокруг объекта ограничивающую рамку, вам понадобятся еще два числа: ширина и высота объекта. Итак, у вас есть четыре выходных нейрона.

В Scikit-Learn есть MLPRegressorкласс, поэтому давайте воспользуемся им для создания MLP с тремя скрытыми слоями, состоящими из 50 нейронов каждый, и обучим его на наборе данных о жилье в Калифорнии.

Следующий код начинается с выборки и разделения набора данных, затем он создает конвейер для стандартизации входных объектов перед их отправкой в ​​файл MLPRegressor. Это очень важно для нейронных сетей, поскольку они обучаются с помощью градиентного спуска, Градиентный спуск не очень хорошо сходится, когда объекты имеют очень разные масштабы. 

Наконец, код обучает модель и оценивает ее ошибку проверки. Модель использует функцию активации ReLU в скрытых слоях и вариант градиентного спуска под названием Adam, чтобы минимизировать среднеквадратическую ошибку, с небольшой регуляризацией ℓ 2 (которой вы можете управлять с помощью alpha гиперпараметра):

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing() # набор данных
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

mlp_reg = MLPRegressor(hidden_layer_sizes=[50, 50, 50], random_state=42) # три скрытых слоя по 50 нейронов
pipeline = make_pipeline(StandardScaler(), mlp_reg) # создаем контейнер с маштабированием и самой моделью
pipeline.fit(X_train, y_train) # обучение
y_pred = pipeline.predict(X_valid) # предсказание
rmse = mean_squared_error(y_valid, y_pred, squared=False)  

In [3]:
rmse

0.5053326657968666

Мы получаем RMSE проверки около 0,505, что сопоставимо с тем, что вы получили бы с помощью классификатора случайного леса.

Типичная архитектура регрессионного MLP:

Гиперпараметр	                           
- скрытых слоев:               Зависит от проблемы, но обычно от 1 до 5.

- нейронов на скрытый слой:    Зависит от проблемы, но обычно от 10 до 100.

- выходных нейронов:           1 на каждое измерение прогноза

- Скрытая активация: РеЛУ

- Активация выхода: Нет, или ReLU/softplus (если выходные данные положительные) или сигмовидная/tanh (если выходные данные ограничены)

- Функция потерь: MSE или Huber, если выбросы

## Классификация МЛП

MLP также можно использовать для задач классификации.Для задачи двоичной классификации вам нужен всего лишь один выходной нейрон с использованием сигмовидной функции активации: выходным сигналом будет число от 0 до 1, которое вы можете интерпретировать как предполагаемую вероятность положительного класса. Предполагаемая вероятность отрицательного класса равна единице минус это число.

MLP также могут легко решать задачи двоичной классификации по нескольким меткам. Например, у вас может быть система классификации электронной почты, которая предсказывает, является ли каждое входящее письмо спамом или нежелательным, и одновременно прогнозирует, является ли оно срочным или несрочным. В этом случае вам понадобятся два выходных нейрона, оба используют сигмовидную функцию активации: первый будет выводить вероятность того, что электронное письмо является спамом, а второй — вероятность того, что оно срочное. 

Если каждый экземпляр может принадлежать только одному классу из трех или более возможных классов (например, классы от 0 до 9 для классификации изображений цифр), то вам необходимо иметь один выходной нейрон для каждого класса, и вам следует использовать функцию активации softmax. для всего выходного слоя.Функция softmax гарантирует, что все оцененные вероятности находятся в диапазоне от 0 до 1 и что их сумма равна 1, поскольку классы являются исключительными. 

Типичная классификационная архитектура MLP

Гиперпараметр	
- скрытых слоев: Обычно от 1 до 5 слоев, в зависимости от задачи.

- выходных нейронов: 1, 1 на двоичную метку, 1 на класс

- Активация выходного слоя: сигмовидная (Бинарная классификация и многозначная бинарная классификация), Софтмакс (Мультиклассовая классификация)

- Функция потерь: X-энтропия

## MLP с Keras